In [7]:
import torch
print('torch版本：',torch.__version__)

torch版本： 2.0.1+cu117


In [9]:
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device',device)

device cuda:0


In [8]:
from torchvision import models

In [19]:
model= models.resnet50(weights="IMAGENET1K_V1").to(device)
model=model.eval().to(device)
x=torch.randn(1,3,224,224).to(device)
output=model(x)
output.shape

torch.Size([1, 1000])

In [21]:
import onnx
print('onnx版本：',onnx.__version__)

onnx版本： 1.14.1


In [22]:
with torch.no_grad():
    torch.onnx.export(
    model,
    x,
    'resnet50_imagenet.onnx',
    opset_version=11,
    input_names=['input'],
    output_names=['output']
    )

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [1]:
import onnx
onnx_model=onnx.load('resnet50_imagenet.onnx')
onnx.checker.check_model(onnx_model)
print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


In [2]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %input[FLOAT, 1x3x224x224]
) initializers (
  %fc.weight[FLOAT, 1000x2048]
  %fc.bias[FLOAT, 1000]
  %onnx::Conv_497[FLOAT, 64x3x7x7]
  %onnx::Conv_498[FLOAT, 64]
  %onnx::Conv_500[FLOAT, 64x64x1x1]
  %onnx::Conv_501[FLOAT, 64]
  %onnx::Conv_503[FLOAT, 64x64x3x3]
  %onnx::Conv_504[FLOAT, 64]
  %onnx::Conv_506[FLOAT, 256x64x1x1]
  %onnx::Conv_507[FLOAT, 256]
  %onnx::Conv_509[FLOAT, 256x64x1x1]
  %onnx::Conv_510[FLOAT, 256]
  %onnx::Conv_512[FLOAT, 64x256x1x1]
  %onnx::Conv_513[FLOAT, 64]
  %onnx::Conv_515[FLOAT, 64x64x3x3]
  %onnx::Conv_516[FLOAT, 64]
  %onnx::Conv_518[FLOAT, 256x64x1x1]
  %onnx::Conv_519[FLOAT, 256]
  %onnx::Conv_521[FLOAT, 64x256x1x1]
  %onnx::Conv_522[FLOAT, 64]
  %onnx::Conv_524[FLOAT, 64x64x3x3]
  %onnx::Conv_525[FLOAT, 64]
  %onnx::Conv_527[FLOAT, 256x64x1x1]
  %onnx::Conv_528[FLOAT, 256]
  %onnx::Conv_530[FLOAT, 128x256x1x1]
  %onnx::Conv_531[FLOAT, 128]
  %onnx::Conv_533[FLOAT, 128x128x3x3]
  %onnx::Conv_534[FLOAT, 128]
  %onnx::Conv_536[FLO

In [10]:
import torch
import onnxruntime
print('onnxruntime版本：',onnxruntime.__version__)

onnxruntime版本： 1.16.0


In [13]:
ort_session=onnxruntime.InferenceSession('resnet50_imagenet.onnx',providers=['TensorrtExecutionProvider','CPUExecutionProvider'])
x=torch.randn(1,3,224,224).numpy()
ort_inputs={'input':x}
ort_output=ort_session.run(['output'],ort_inputs)[0]
ort_output.shape

(1, 1000)